In [1]:
import os
import glob #untuk mengambil semua file dalam satu folder
import requests #supaya bisa mengambil data dari website dengan cara request
import pandas as pd
import itertools #untuk menggabungkan dua list
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup #beautifulsoup untuk mengambil data dari website dengan cara parsing atau mengurai data html
from selenium import webdriver # webdriver bisa mengambil data yang dihasilkan oleh javascript


In [2]:
def scrapweb(var_list, awal_halaman, akhir_halaman):
    header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0'}
    halaman = awal_halaman
    while halaman <= akhir_halaman:
        url = f'https://www.lamudi.co.id/west-java/bekasi/house/buy/?page={halaman}'
        req = requests.get(url, headers=header, timeout=10)
        soup = BeautifulSoup(req.content, 'html.parser')
        semua_tempat = soup.find_all('div', class_='ListingCell-AllInfo ListingUnit')

        for i in semua_tempat:
            try:
                nama = i.find('h3', class_='ListingCell-KeyInfo-title')['title']
            except (AttributeError, KeyError):
                nama = None
            try:
                alamat = i.find('div', class_='ListingCell-KeyInfo-address ellipsis').find('span', class_='ListingCell-KeyInfo-address-text').text.strip()
            except (AttributeError, KeyError):
                alamat = None
            try:
                info_key = i.find('div', class_='KeyInformation_v2')
            
                # Mengambil semua div dengan class KeyInformation-attribute_v2
                attribute_divs = info_key.find_all('div', class_='KeyInformation-attribute_v2')
            
                # Mengambil nilai dari setiap atribut sesuai dengan labelnya
                for div in attribute_divs:
                    label = div.find('span', class_='KeyInformation-label_v2').text.strip()
                    value = div.find('span', class_='KeyInformation-value_v2').text.strip()
            
                    if label == 'Kamar tidur':
                        kamar_tidur = value
                    elif label == 'Bangunan':
                        luas_bangunan = value
                    elif label == 'Lahan':
                        luas_tanah = value
            
            except AttributeError:
                kamar_tidur = None
                luas_bangunan = None
                luas_tanah = None
            try:
                harga = i.find('div', class_='ListingCell-KeyInfo-PriceWrapper').find('span', class_='PriceSection-FirstPrice').text.strip()
            except (AttributeError, KeyError):
                harga = None
            try:
                deskripsi = i.find('div', class_='ListingCell-shortDescription').text.strip()
            except (AttributeError, KeyError):
                deskripsi = None
            var_list.append({
                'Nama': nama,
                'Alamat': alamat,
                'Kamar Tidur': kamar_tidur,
                'Luas Bangunan': luas_bangunan,
                'Luas Tanah': luas_tanah,
                'Harga': harga,
                'Deskripsi': deskripsi
            })
        halaman += 1

In [3]:
test = []
scrapweb(test, 1, 1)
len(test)

30

In [4]:
df = pd.DataFrame(test)
df

,Nama,Alamat,Kamar Tidur,Luas Bangunan,Luas Tanah,Harga,Deskripsi
0,Dijual Rumah Strategis Bebas Banjir Pondok Ung...,"Babelan, Bekasi",2,100 m²,60 m²,Rp560.000.000,Pondok Ungu Permai .Blok AD16 (J-17533) Pondok...
1,Rumah Bintara 2 Lantai Lokasi Baru & Strategis,"Bintara Jaya, Bekasi",3,90 m²,60 m²,Rp825.000.000,Di Jual Rumah Bintara Jaya Dekat Ke LRT Cikuni...
2,Rumah 3 Lantai Strategis di Kota Harapan Indah...,"Harapan Indah, Bekasi",4,250 m²,163 m²,Rp2.040.000.000,"Dijual Cepat Rumah siap huni lokasi strategis,..."
3,Jual Cepat rumah di Harapan indah Bekasi Harga...,"Harapan Indah, Bekasi",2,84 m²,94 m²,Rp755.000.000,Lokasi sangat strategis : § Dekat dengan Pusat...
4,Dijual Rumah Siap Huni Tanah Luas di Kota Beka...,"Harapan Indah, Bekasi",3,95 m²,128 m²,Rp860.000.000,Perumahan Graha Harapan (J-15927) MUSTIKAJAYA...
5,Rumah Nyaman di Harapan Indah,"Harapan Indah, Bekasi",3,112 m²,112 m²,Rp1.550.000.000,"Dijual Rumah di Harapan Indah, Bekasi SHM Lt 1..."
6,Di jual rumah cantik di aralia siap pakai Hara...,"Harapan Indah, Bekasi",4,160 m²,160 m²,Rp2.000.000.000,Di jual rumah cantik di aralia siap pakai Hara...
7,"KOMERSIL PROMO GARDENIA CITY 3 CIBITUNG,","Cibitung, Bekasi",2,36 m²,90 m²,Rp352.000.000,Gardenia City 3 Lokasi CIBITUNG open untuk kom...
8,Dijual Rumah 3 Lt.Lokasi Strategis diapit 2 Tol,"Pondok Gede, Bekasi",5,450 m²,321 m²,Rp3.400.000.000,Dijual Rumah Antik Lokasi Strategis diapit 3 A...
9,GRIYA BAGASASI CIKARANG SUKATANI,"Cikarang Utara, Bekasi",2,90 m²,123 m²,Rp375.000.000,JUAL CASH atau KPR Perumahan Griya Bagasasi Bl...


In [5]:
objek5 = []
scrapweb(objek5, 1, 50)
len(objek5)

1500

In [6]:
df6 = pd.DataFrame(objek5)
df6

,Nama,Alamat,Kamar Tidur,Luas Bangunan,Luas Tanah,Harga,Deskripsi
0,Dijual Rumah Strategis Bebas Banjir Pondok Ung...,"Babelan, Bekasi",2,100 m²,60 m²,Rp560.000.000,Pondok Ungu Permai .Blok AD16 (J-17533) Pondok...
1,Rumah Bintara 2 Lantai Lokasi Baru & Strategis,"Bintara Jaya, Bekasi",3,90 m²,60 m²,Rp825.000.000,Di Jual Rumah Bintara Jaya Dekat Ke LRT Cikuni...
2,Rumah 3 Lantai Strategis di Kota Harapan Indah...,"Harapan Indah, Bekasi",4,250 m²,163 m²,Rp2.040.000.000,"Dijual Cepat Rumah siap huni lokasi strategis,..."
3,Jual Cepat rumah di Harapan indah Bekasi Harga...,"Harapan Indah, Bekasi",2,84 m²,94 m²,Rp755.000.000,Lokasi sangat strategis : § Dekat dengan Pusat...
4,Dijual Rumah Siap Huni Tanah Luas di Kota Beka...,"Harapan Indah, Bekasi",3,95 m²,128 m²,Rp860.000.000,Perumahan Graha Harapan (J-15927) MUSTIKAJAYA...
...,...,...,...,...,...,...,...
1495,rmh lega mewah DIBINTARA,"Bintara, Bekasi",5,220 m²,145 m²,Rp2.000.000.000,Di jual rumah di bintara dalam cluster.Rumah 2...
1496,Rumah 2 Lantai di Lippo Cikarang-Cendana Spark...,"Cikarang Selatan, Bekasi",3,68 m²,83 m²,Rp1.065.000.000,Spark Residence (Cendana Spark Phase 2) yang b...
1497,Dijual Rumah Mustika Jaya Cimuning Bekasi Timur,"Mustika Jaya, Bekasi",2,58 m²,84 m²,Rp697.000.000,Cluster Exclusive hanya 60 unit saja di Cimuni...
1498,rumah jual dicibubur country,"Cibubur , Bekasi",1,28 m²,179 m²,Rp975.000.000,"rumah lb 28m, lt 179m, aman dalam kluster."


In [ ]:
path = 'dataset/'

# Nama file CSV yang diinginkan
nama_file = 'bekasi.csv'

# Gabungkan path dan nama file
full_path = path + nama_file

# Simpan dataframe ke dalam file CSV
df6.to_csv(full_path, index=False)

print(f"Data berhasil disimpan ke: {full_path}")

In [6]:
test = []
scrapweb(test, 1, 1)

In [7]:
len(test)

20

In [8]:
objek1 = []
objek2 = []
objek3 = []
objek4 = []
objek5 = []
objek6 = []
objek7 = []
objek8 = []
objek9 = []
objek10 = []

In [9]:
scrapweb(objek1, 1, 20)

In [10]:
len(objek1)

20

In [11]:
df = pd.DataFrame(objek1)
df

,Nama Tempat,Harga,Alamat,Cicilan,Kamar Tidur,Kamar Mandi,Garasi,Luas Tanah,Luas Bangunan
0,Rumah 5 Menit Ke Tol Pasteur Minimalis Aman Ny...,Rp 975 Juta,"Cimahi Utara, Cimahi",Cicilan: 3 Jutaan per bulan,3,2,None,110 m²,86 m²
1,Cimahi Utara Home Industri Nusa Sari Rumah Bes...,"Rp 1,69 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,2,2,2,200 m²,180 m²
2,Dijual Cepat Dibulan Penuh Berkah Harga Nego R...,Rp 645 Juta,"Cimahi Selatan, Cimahi",Cicilan: 2 Jutaan per bulan,6,5,None,64 m²,128 m²
3,Rumah Asri 2 Lantai Dalam Komplek Strategis Di...,"Rp 2,6 Miliar","Cimahi Utara, Cimahi",Cicilan: 9 Jutaan per bulan,6,4,3,304 m²,250 m²
4,"Rumah Lelang Cimahi, Jawa Barat",Rp 385 Juta,"Cimahi Selatan, Cimahi",Cicilan: 1 Jutaan per bulan,1,1,None,72 m²,38 m²
5,Rumah 1.5 Lantai Permata Cimahi Butuh mendesak...,Rp 399 Juta,"Cimahi Utara, Cimahi",Cicilan: 1 Jutaan per bulan,2,1,1,60 m²,70 m²
6,Rumah di Jalan Permana Cimahi Utara 2 Lantai SHM,"Rp 1,7 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,3,2,1,171 m²,160 m²
7,Rumah Bagus 2 Lantai Komplek Taman Mutiara Cim...,"Rp 1,39 Miliar","Cimahi Tengah, Cimahi",Cicilan: 5 Jutaan per bulan,3,3,None,100 m²,170 m²
8,Sikat Hangatt Rumah di Taman Bumi Prima Cimahi...,"Rp 1,55 Miliar","Cimahi Utara, Cimahi",Cicilan: 5 Jutaan per bulan,3,2,1,203 m²,100 m²
9,Langka Gaada Lagii Rumah Di Taman Mutiara Cima...,"Rp 1,17 Miliar","Cimahi Tengah, Cimahi",Cicilan: 4 Jutaan per bulan,2,1,1,141 m²,70 m²


In [8]:
# Membuat DataFrame dari data_list
df = pd.DataFrame(test)

In [9]:
path = '/workspaces/codespaces-jupyter/cimahi/simpan/'
csv_path = os.path.join(path, 'Jual_rumah_cimahi_tes.csv')
df.to_csv(csv_path, index=False)  # index=False untuk menghilangkan index tambahan

# Memastikan header hanya ditambahkan sekali
df2 = pd.read_csv(csv_path)
df2

,Nama Tempat,Harga,Alamat,Cicilan,Kamar Tidur,Kamar Mandi,Garasi,Luas Tanah,Luas Bangunan
0,Cimahi Utara Home Industri Nusa Sari Rumah Bes...,"Rp 1,69 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,2,2,2.0,200 m²,180 m²
1,Dijual Cepat Dibulan Penuh Berkah Harga Nego R...,Rp 645 Juta,"Cimahi Selatan, Cimahi",Cicilan: 2 Jutaan per bulan,6,5,NaN,64 m²,128 m²
2,Rumah Asri 2 Lantai Dalam Komplek Strategis Di...,"Rp 2,6 Miliar","Cimahi Utara, Cimahi",Cicilan: 9 Jutaan per bulan,6,4,3.0,304 m²,250 m²
3,"Rumah Lelang Cimahi, Jawa Barat",Rp 385 Juta,"Cimahi Selatan, Cimahi",Cicilan: 1 Jutaan per bulan,1,1,NaN,72 m²,38 m²
4,Rumah Kost 3 Lantai di Kampung Ranca Cangkuang...,Rp 900 Juta,"Cimahi Selatan, Cimahi",Cicilan: 3 Jutaan per bulan,15,6,NaN,115 m²,90 m²
5,Rumah 1.5 Lantai Permata Cimahi Butuh mendesak...,Rp 399 Juta,"Cimahi Utara, Cimahi",Cicilan: 1 Jutaan per bulan,2,1,1.0,60 m²,70 m²
6,Rumah di Jalan Permana Cimahi Utara 2 Lantai SHM,"Rp 1,7 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,3,2,1.0,171 m²,160 m²
7,Rumah 5 Menit Ke Tol Pasteur Minimalis Aman Ny...,Rp 975 Juta,"Cimahi Utara, Cimahi",Cicilan: 3 Jutaan per bulan,3,2,NaN,110 m²,86 m²
8,RUMAH CIPAGERAN CIMAHI CEPAT,Rp 650 Juta,"Cimahi Utara, Cimahi",Cicilan: 2 Jutaan per bulan,3,1,NaN,71 m²,72 m²
9,Murah! Dijual rumah minimalis dalam komplek Ci...,Rp 490 Juta,"Cimahi Utara, Cimahi",Cicilan: 1 Jutaan per bulan,2,1,1.0,72 m²,55 m²
